In [3]:
import os
os.chdir('../utils/')
from UtilityFunctions import read_corrs, read_bed_dat_feat
import pylab as P
import numpy as np

path = '/Users/elizabethpaul/Documents/COS424/beds/'
train = read_bed_dat_feat(path, chrom=1)
corrs = read_corrs(path, chrom=1)

In [7]:
os.chdir('/Users/elizabethpaul/Documents/COS424/')
# Trim off nans
corrs = corrs[1:-1]
train = train[1:-1]
# Plot distribution
P.hist(corrs, bins=30)
P.title('All Sites')
P.savefig('corr_all_chr1.png')
# Plot distribution of CGI corrs
corrs_cgi = corrs[train['CGI'] == 1]
P.figure()
P.hist(corrs_cgi, bins=30)
P.title('CGI')
P.savefig('corr_CGI_chr1.png')
# Plot distribution of exon corrs
corrs_exon = corrs[train['Exon'] == 1]
P.figure()
P.hist(corrs_exon, bins=30)
P.title('Exon')
P.savefig('corr_Exon_chr1.png')
# Plot distribution of DHS regions corrs
corrs_dhs = corrs[train['DHS'] > 0]
P.figure()
P.hist(corrs_dhs, bins=30)
P.title('DHS')
P.savefig('corr_DHS_chr1.png')
P.show()

In [13]:
beta = train['Beta'][:,0]
# Plot distribution
P.hist(beta, bins=30, range =(0, 1))
P.savefig('beta_all_chr1.png')
# Plot distribution of CGI beta
beta_cgi = beta[train['CGI'] == 1]
P.figure()
P.hist(beta_cgi, bins=30, range =(0, 1))
P.title('CGI')
P.savefig('beta_CGI_chr1.png')
# Plot distribution of exon corrs
beta_exon = beta[train['Exon'] == 1]
P.figure()
P.hist(beta_exon, bins=30, range =(0, 1))
P.title('Exon')
P.savefig('beta_Exon_chr1.png')
# Plot distribution of DHS regions corrs
beta_dhs = beta[train['DHS'] > 0]
P.figure()
P.hist(beta_dhs, bins=30, range =(0, 1))
P.title('DHS')
P.savefig('beta_DHS_chr1.png')
P.show()

In [16]:
sample = read_bed_dat_feat(path, chrom=1, ftype='sample')
test = read_bed_dat_feat(path, chrom=1, ftype='test')
train = read_bed_dat_feat(path, chrom=1, ftype='train')

In [18]:
os.chdir('/Users/elizabethpaul/Documents/COS424/COS-424-HW-2/models')
from RandomForest import *
os.chdir('../utils/')
from UtilityFunctions import calc_r2_RMSE

In [ ]:
sites = train['Start']
train_extras = np.c_[train['Exon'], train['DHS'], train['CGI']]
train_extras = np.c_[train_extras, train['GC_100'], train['GC_400'], train['GC_1000']]
train_extras = np.c_[train_extras, uf.read_corrs(path, chroms)]


In [ ]:
train_beta = fill_mean(train['Beta'])
thresh = 0.5
(X, Y, Xstar, Ystar) = feat_neighbors_thresh(sites[1:-1].copy(), train_beta[1:-1].copy(), \
train_extras[1:-1].copy(), sample[1:-1].copy(), test[1:-1].copy(), uf.read_corrs(path, chroms), thresh)


In [ ]:
trees = 30
full_feat_start = time.time()
model = RandomForestRegressor(oob_score=True, n_estimators=trees)
model.fit(X, Y)
Yhat = model.predict(Xstar)
(r2, RMSE) = calc_r2_RMSE(Yhat, Ystar)